In [1]:
# load packages
import pandas as pd

In [28]:
# load eid_data
pob_data = pd.read_csv('../where_do_kin_live/10km_grid_pob_centroids.csv')
print(len(pob_data))
pob_data.head(3)

442035


,eid,center,DOB,POB_north,POB_east,NUTS_ID,NUTS_NAME,grid_id,X,Y
0,1136349,11004,1947,801500,68500,UKM64,Eilean Siar (Western Isles),774,64917.76409,803641.428192
1,1071275,11004,1952,798500,66500,UKM64,Eilean Siar (Western Isles),775,64917.76409,793641.428192
2,4778319,11005,1946,798500,66500,UKM64,Eilean Siar (Western Isles),775,64917.76409,793641.428192


In [29]:
pob_data[pob_data.duplicated(subset=['eid'], keep = False)]

,eid,center,DOB,POB_north,POB_east,NUTS_ID,NUTS_NAME,grid_id,X,Y


# add place of residence (POR) data to eid_data and save a new version here

In [30]:
# load place of residence (POR) data
por_data = pd.read_csv('../por_healthdata/por_locations.csv')
print(len(por_data))
por_data.head(2)

502536


,eid,22702-0.0,22704-0.0
0,3609487,330000.0,407000.0
1,1194904,378000.0,200000.0


In [31]:
# rename columns in por_data
por_data.rename(index=str, columns={"22702-0.0": "POR_east", "22704-0.0": "POR_north"},inplace=True)
por_data.head(2)

,eid,POR_east,POR_north
0,3609487,330000.0,407000.0
1,1194904,378000.0,200000.0


In [32]:
pob_data.head(2)

,eid,center,DOB,POB_north,POB_east,NUTS_ID,NUTS_NAME,grid_id,X,Y
0,1136349,11004,1947,801500,68500,UKM64,Eilean Siar (Western Isles),774,64917.76409,803641.428192
1,1071275,11004,1952,798500,66500,UKM64,Eilean Siar (Western Isles),775,64917.76409,793641.428192


In [33]:
# rename columns in pob_data
pob_data.rename(index=str, columns={"NUTS_ID": "POB_NUTSID", "grid_id": "POB_gridID",
                                   "X": "POB_gridX", "Y": "POB_gridY" },inplace=True)
pob_data.head(2)

,eid,center,DOB,POB_north,POB_east,POB_NUTSID,NUTS_NAME,POB_gridID,POB_gridX,POB_gridY
0,1136349,11004,1947,801500,68500,UKM64,Eilean Siar (Western Isles),774,64917.76409,803641.428192
1,1071275,11004,1952,798500,66500,UKM64,Eilean Siar (Western Isles),775,64917.76409,793641.428192


In [39]:
pob_data[pob_data.duplicated(subset=['eid'], keep = False)]

,eid,center,DOB,POB_north,POB_east,POB_NUTSID,NUTS_NAME,POB_gridID,POB_gridX,POB_gridY


In [35]:
# merge pob_data and por_data into eid_data
eid_data = pob_data.merge(por_data, on = 'eid')

In [38]:
print(len(pob_data))
print(len(por_data))
print(len(eid_data))
eid_data.head(2)

442035
502536
441943


,eid,center,DOB,POB_north,POB_east,POB_NUTSID,NUTS_NAME,POB_gridID,POB_gridX,POB_gridY,POR_east,POR_north
0,1136349,11004,1947,801500,68500,UKM64,Eilean Siar (Western Isles),774,64917.76409,803641.428192,252000.0,663000.0
1,1071275,11004,1952,798500,66500,UKM64,Eilean Siar (Western Isles),775,64917.76409,793641.428192,211000.0,665000.0


In [40]:
# drop NUTS_NAME
# df.drop(columns=['B', 'C'])
eid_data.drop(columns = 'NUTS_NAME', inplace = True)
eid_data.head(5)

,eid,center,DOB,POB_north,POB_east,POB_NUTSID,POB_gridID,POB_gridX,POB_gridY,POR_east,POR_north
0,1136349,11004,1947,801500,68500,UKM64,774,64917.76409,803641.428192,252000.0,663000.0
1,1071275,11004,1952,798500,66500,UKM64,775,64917.76409,793641.428192,211000.0,665000.0
2,4778319,11005,1946,798500,66500,UKM64,775,64917.76409,793641.428192,323000.0,669000.0
3,2797312,11011,1949,842500,79500,UKM64,892,74917.76409,843641.428192,358000.0,171000.0
4,3735126,11002,1948,842500,79500,UKM64,892,74917.76409,843641.428192,459000.0,205000.0


In [41]:
print(len(eid_data))
eid_data[eid_data.duplicated(subset=['eid'],keep=False)]

441943


,eid,center,DOB,POB_north,POB_east,POB_NUTSID,POB_gridID,POB_gridX,POB_gridY,POR_east,POR_north


In [14]:
# save eid_data here
# eid_data.to_csv('eid_data.csv',index=False)`

In [17]:
# off to QGIS to add POR_NUTSID and do grid assignments ... grid might be a bit rough!

NUTS3 was a bit of a challenge because 1566 coastal/river points had no NUTS3 assignment.  
With QGIS and buffer, was able to get them all.

In [20]:
nuts3_por_1stPass = pd.read_csv('../por_healthdata/nuts3_por_1st_pass.csv')

In [42]:
print(len(nuts3_por_1stPass))
nuts3_por_1stPass.head(2)

440377


,eid,NUTS_ID,NUTS_NAME
0,4984208,UKC11,Hartlepool and Stockton-on-Tees
1,1104285,UKC11,Hartlepool and Stockton-on-Tees


In [22]:
nuts3_por_1stPass[nuts3_por_1stPass.duplicated(subset=['eid'],keep=False)]

,eid,NUTS_ID,NUTS_NAME


In [25]:
noData = nuts3_por_1stPass[nuts3_por_1stPass.isnull().any(axis=1)].index.tolist()
dropped = nuts3_por_1stPass.drop(nuts3_por_1stPass.index[noData])
print('nuts3_por_1stPass',len(nuts3_por_1stPass))
print('number with null data',len(noData))
print('null data removed',len(dropped))

0
nuts3_por_1stPass 440377
number with null data 0
null data removed 440377


In [9]:
# save new version of nuts3_por_1stPass with null data removed
!mv nuts3_por_1st_pass.csv nuts3_por_1st_pass_withNullData.csv
dropped.to_csv('nuts3_por_1st_pass.csv',index=False)

In [44]:
# get the points that still need nuts3 ids
old = pd.read_csv('../por_healthdata/nuts3_por_1st_pass_withNullData.csv')

In [52]:
noData = old[old.isnull().any(axis=1)]
print(len(noData))
print(len(nuts3_por_1stPass))
print(len(noData) + len(nuts3_por_1stPass))
print(len(eid_data))

1566
440377
441943
441943


In [46]:
noData.head(2)

,eid,NUTS_ID,NUTS_NAME
440377,1071275,NaN,NaN
440378,3697624,NaN,NaN


In [8]:
eid_data = pd.read_csv('eid_data.csv')

In [9]:
p = noData.merge(eid_data, on='eid')

In [12]:
p[['eid','POR_east','POR_north']].to_csv('needNuts3.csv',index=False)

In [13]:
noData[noData['eid']==3348493]

,eid,NUTS_ID,NUTS_NAME
440960,3348493,NaN,NaN


In [50]:
# OK i have them all done ... merge them in terminal via cat
# nuts3_por_1st_pass.csv + nuts3_por_2nd_pass.csv
# PROBLEM!!!!!! nuts3_por_2nd_pass.csv has duplicate data. need to fix and save a new version of eid_data
# I did this in eid_list_for_pobi
all_por = pd.read_csv('../por_healthdata/nuts3_por_all.csv')

In [55]:
# PROBLEM!!!! There are duplicates!!!!
all_por[all_por.duplicated(subset=['eid'],keep=False)].sort_values(by='eid')

,eid,NUTS_ID,NUTS_NAME
441674,1000708,UKI23,Outer London - West and North West
441220,1000708,UKI11,Inner London - West
441310,1023392,UKI12,Inner London - East
441002,1023392,UKI11,Inner London - West
442353,1027080,UKM35,"Inverclyde, East Renfrewshire and Renfrewshire"
442317,1027080,UKM34,Glasgow City
441515,1032029,UKI23,Outer London - West and North West
440943,1032029,UKI11,Inner London - West
441161,1034229,UKI11,Inner London - West
441652,1034229,UKI23,Outer London - West and North West


In [56]:
print(len(all_por))
all_por.head(3)

442400


,eid,NUTS_ID,NUTS_NAME
0,4984208,UKC11,Hartlepool and Stockton-on-Tees
1,1104285,UKC11,Hartlepool and Stockton-on-Tees
2,2998376,UKC11,Hartlepool and Stockton-on-Tees


In [57]:
# rename columns in all_por
all_por.rename(index=str, columns={"NUTS_ID": "POR_NUTSID"},inplace=True)
all_por.head(2)

,eid,POR_NUTSID,NUTS_NAME
0,4984208,UKC11,Hartlepool and Stockton-on-Tees
1,1104285,UKC11,Hartlepool and Stockton-on-Tees


In [58]:
eid_data.head(2)

,eid,center,DOB,POB_north,POB_east,POB_NUTSID,POB_gridID,POB_gridX,POB_gridY,POR_east,POR_north
0,1136349,11004,1947,801500,68500,UKM64,774,64917.76409,803641.428192,252000.0,663000.0
1,1071275,11004,1952,798500,66500,UKM64,775,64917.76409,793641.428192,211000.0,665000.0


In [16]:
# merge all_por with eid_data
got_them_all = eid_data.merge(all_por[['eid','POR_NUTSID']],on='eid')

In [17]:
got_them_all.head(2)

,eid,center,DOB,POB_north,POB_east,POB_NUTSID,POB_gridID,POB_gridX,POB_gridY,POR_east,POR_north,POR_NUTSID
0,1136349,11004,1947,801500,68500,UKM64,774,64917.76409,803641.428192,252000.0,663000.0,UKM34
1,1071275,11004,1952,798500,66500,UKM64,775,64917.76409,793641.428192,211000.0,665000.0,UKM63


In [18]:
got_them_all[got_them_all.duplicated(subset=['eid'],keep=False)]

,eid,center,DOB,POB_north,POB_east,POB_NUTSID,POB_gridID,POB_gridX,POB_gridY,POR_east,POR_north,POR_NUTSID
1112,5542874,11018,1957,32500,180500,UKK30,2315,184917.76409,33641.428192,524000.0,176000.0,UKI11
1113,5542874,11018,1957,32500,180500,UKK30,2315,184917.76409,33641.428192,524000.0,176000.0,UKI23
1272,3690842,11012,1947,215500,195500,UKL14,2419,194917.76409,213641.428192,530000.0,178000.0,UKI11
1273,3690842,11012,1947,215500,195500,UKL14,2419,194917.76409,213641.428192,530000.0,178000.0,UKI12
1925,5296400,11018,1944,676500,217500,UKM63,2617,214917.76409,673641.428192,524000.0,176000.0,UKI11
1926,5296400,11018,1944,676500,217500,UKM63,2617,214917.76409,673641.428192,524000.0,176000.0,UKI23
2344,1589535,11012,1955,676500,227500,UKM35,2739,224917.76409,673641.428192,534000.0,180000.0,UKI11
2345,1589535,11012,1955,676500,227500,UKM35,2739,224917.76409,673641.428192,534000.0,180000.0,UKI12
2720,4232289,11004,1965,642500,223500,UKM33,2742,224917.76409,643641.428192,253000.0,667000.0,UKM34
2721,4232289,11004,1965,642500,223500,UKM33,2742,224917.76409,643641.428192,253000.0,667000.0,UKM35


In [30]:
# save new version of eid_data
got_them_all.to_csv('eid_data.csv',index=False)

Now, off to get the grid IDs for POR.  